# 完全不一样

In [ ]:
import pandas as pd

from typing import Optional, Union
import pandas as pd, numpy as np, torch
from dataclasses import dataclass
from transformers import (
    AutoTokenizer,
    AutoConfig,
    EarlyStoppingCallback,
    AutoModelForCausalLM,
    AutoModelForMultipleChoice,
    TrainingArguments,
    Trainer,
    RobertaForMultipleChoice,
    AutoModelForSequenceClassification,
    LlamaModel,
    LlamaForSequenceClassification,
    BitsAndBytesConfig,
    get_polynomial_decay_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    TrainerCallback,
)
import argparse
from transformers import get_polynomial_decay_schedule_with_warmup, TrainerCallback
import datasets
from datasets import Dataset
from random import random, randint
from utils import load_json, load_split_data

import random
def seed_everything(seed=None):
    '''
    固定seed
    :param seed: int, 随机种子
    '''
    max_seed_value = np.iinfo(np.uint32).max
    min_seed_value = np.iinfo(np.uint32).min

    if (seed is None) or not (min_seed_value <= seed <= max_seed_value):
        seed = random.randint(np.iinfo(np.uint32).min, np.iinfo(np.uint32).max)
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    return seed

In [ ]:
data = pd.read_csv('dataset/train.csv')
data = load_json(data)

ex_33 = pd.read_csv('dataset/lmsys-chatbot_arena_conversations-33k.csv')
ex_33 = load_json(ex_33)

In [ ]:
'''
1、找出train里面不与33k重复部分
2、不重复的部分再划分
'''
set_prompt_response = []
for i in data.itertuples():
    prompt_response = i.prompt + i.response_a + i.response_b
    set_prompt_response.append(set(prompt_response))
data['set_prompt_response'] = set_prompt_response    

In [ ]:
set_prompt_response = []
for i in ex_33.itertuples():
    prompt_response = i.prompt + i.response_a + i.response_b
    set_prompt_response.append(set(prompt_response))
ex_33['set_prompt_response'] = set_prompt_response  

In [ ]:
idx = [idx for idx, i in enumerate(data.set_prompt_response.values) if i in ex_33.set_prompt_response.values]

In [ ]:
same = data.loc[idx,:].reset_index(drop = True)
not_same = data.loc[~data.index.isin(idx),:].reset_index(drop = True)

In [ ]:
assert len(not_same) + len(same) == len(data)

In [ ]:
unique_sets = not_same['set_prompt_response'].drop_duplicates().reset_index(drop=True)
# 将唯一集合进行随机划分
unique_sets = unique_sets.sample(frac=1, random_state=42).reset_index(drop=True)
midpoint = len(unique_sets) // 10
set1 = unique_sets.iloc[:midpoint]
set2 = unique_sets.iloc[midpoint:]

In [ ]:
# 根据划分结果从原数据集中提取对应的行
valid = not_same[not_same['set_prompt_response'].isin(set1)].reset_index(drop=True)
train_subset = not_same[not_same['set_prompt_response'].isin(set2)].reset_index(drop=True)
assert len(valid) + len(train_subset) == len(not_same)
assert len(valid) + len(train_subset) + len(same) == len(data)

In [ ]:
train_exclude_valid = pd.concat([train_subset, same]).reset_index(drop=True) #train 里面排除valid
len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_exclude_valid.set_prompt_response.values])
assert len(valid) + len(train_exclude_valid) == len(data)

In [ ]:
train_33k = pd.concat([train_subset, ex_33]).reset_index(drop=True)

In [ ]:
train_subset = train_subset.drop(columns = ['set_prompt_response'])
valid = valid.drop(columns = ['set_prompt_response'])
train_exclude_valid = train_exclude_valid.drop(columns = ['set_prompt_response'])
train_33k = train_33k.drop(columns = ['set_prompt_response'])

In [ ]:
train_subset.to_json("dataset/non_overlap/train_subset.json", index = False)
valid.to_json("dataset/non_overlap/valid.json", index = False)
train_exclude_valid.to_json("dataset/non_overlap/train_exclude_valid.json", index = False)
train_33k.to_json("dataset/non_overlap/train_33k.json", index = False)

In [ ]:
# 检查
train_subset = pd.read_json("dataset/non_overlap/train_subset.json")
valid = pd.read_json("dataset/non_overlap/valid.json")
train_exclude_valid = pd.read_json("dataset/non_overlap/train_exclude_valid.json")
train_33k = pd.read_json("dataset/non_overlap/train_33k.json")

In [ ]:
def get_set_prompt_response(data):
    set_prompt_response = []
    for i in data.itertuples():
        prompt_response = i.prompt + i.response_a + i.response_b
        set_prompt_response.append(set(prompt_response))
    data['set_prompt_response'] = set_prompt_response  
    return data

In [ ]:
train_subset = get_set_prompt_response(train_subset)
valid = get_set_prompt_response(valid)
train_exclude_valid = get_set_prompt_response(train_exclude_valid)
train_33k = get_set_prompt_response(train_33k)

In [ ]:
#valid和任何都不重合
assert len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_subset.set_prompt_response.values]) == 0
assert len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_exclude_valid.set_prompt_response.values]) == 0
assert len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_33k.set_prompt_response.values]) == 0

In [ ]:
train_33k.prompt.values[0][1]

# 1M 与 train_33k和valid重复的

In [ ]:
def get_set_prompt_response(data):
    set_prompt_response = []
    for i in data.itertuples():
        prompt_response = i.prompt + i.response_a + i.response_b
        set_prompt_response.append(set(prompt_response))
    data['set_prompt_response'] = set_prompt_response  
    return data

In [ ]:
# 检查
train_subset = pd.read_json("dataset/non_overlap/train_subset.json")
valid = pd.read_json("dataset/non_overlap/valid.json")
train_exclude_valid = pd.read_json("dataset/non_overlap/train_exclude_valid.json")
train_33k = pd.read_json("dataset/non_overlap/train_33k.json")
ex_data = pd.read_json("dataset/1M/35k_in_1M.json")
ex_data = pd.read_parquet("dataset/1M/filtered_df.parquet")

In [ ]:
def drop_dup_hate(data):
    data = data.loc[data.model_a != data.model_b].reset_index(drop = True)
    set_prompt_response = []
    for i in data.itertuples():
        prompt_response = i.prompt + i.response_a + i.response_b
        set_prompt_response.append(set(prompt_response))
    data['set_prompt_response'] = set_prompt_response  
    idx = (data.response_a_categories != 'Good') | (data.response_b_categories != 'Good')
    data = data[~idx].reset_index(drop = True)
    data = data.drop(columns = ['set_prompt_response'])
    return data

In [ ]:
ex_data = drop_dup_hate(ex_data)

In [ ]:
train_subset = get_set_prompt_response(train_subset)
valid = get_set_prompt_response(valid)
train_exclude_valid = get_set_prompt_response(train_exclude_valid)
train_33k = get_set_prompt_response(train_33k)
ex_data = get_set_prompt_response(ex_data)

In [ ]:
len([idx for idx, i in enumerate(ex_data.set_prompt_response.values) if i in train_subset.set_prompt_response.values])

In [ ]:
in_valid = [idx for idx, i in enumerate(ex_data.set_prompt_response.values) if i in valid.set_prompt_response.values]

In [ ]:
len(in_valid)

In [ ]:
ex_data = ex_data.loc[~ex_data.index.isin(in_valid)].reset_index(drop = True)

In [ ]:
ex_data.to_json("dataset/1M/3k_high_quality_method_2.json", index = False)

In [ ]:
ex_data

In [ ]:
len([idx for idx, i in enumerate(ex_data.set_prompt_response.values) if i in train_exclude_valid.set_prompt_response.values])

# 1M 伪标签

In [ ]:
final = pd.read_json("dataset/persudo_label/35k_in_1M_prediction.json")

In [ ]:
threshold1 = 0.75
filter_list = (final.winner_model_a >= threshold1) | (final.winner_model_b >= threshold1) | (final.winner_tie >= threshold1)
filter = final.loc[filter_list,:].reset_index(drop = True)
filter

In [ ]:
def get_label(row):
    val = [row[option] for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie'])]
    label = val.index(max(val))
    #label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    if label == 0:
        return 'A'
    elif label == 1:
        return 'B'
    else:
        return 'C'
    return label

In [ ]:
filter['label'] = filter.apply(lambda x: get_label(x), axis = 1)
filter["winner_model_a"] = 0
filter["winner_model_b"] = 0
filter["winner_tie"] = 0

In [ ]:
filter.loc[filter.label == 'A',"winner_model_a"] = 1
filter.loc[filter.label == 'B',"winner_model_b"] = 1
filter.loc[filter.label == 'C',"winner_tie"] = 1

In [ ]:
filter = filter.drop(columns = ['label'])

In [ ]:
filter

In [ ]:
filter.to_json('dataset/persudo_label/35k_in_1M_prediction_thr75_10k.json', index = False)

In [ ]:
i = 2846
print(filter.loc[i,'prompt'][0])

In [ ]:
print(filter.loc[i,'response_a'][0])

In [ ]:
print(filter.loc[i,'response_b'][0])